In [38]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision.models import resnet50, ResNet50_Weights

import numpy as np
import pandas as pd
from fastparquet import write

import os
import glob as glob

import cv2
from PIL import Image


%matplotlib inline

In [2]:
class FinalLayer(nn.Module):
    """Modified last layer for resnet50 for your dataset"""
    def __init__(self):
        super(FinalLayer, self).__init__()
        self.fc = nn.Linear(2048, 12)  # Assuming you have 12 output classes
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        out = self.fc(x)
        out = self.sigmoid(out)
        return out

def modified_resnet50():
    # Load pretrained resnet50 with a modified last fully connected layer
    model = resnet50(weights=ResNet50_Weights.DEFAULT)
    model.fc = FinalLayer()
    return model

# Load the modified ResNet-50 model
model = modified_resnet50()

# Load the protest prediction model
model_checkpoint = torch.load('../../protest-detection-violence-estimation/model_best.pth.tar')
model.load_state_dict(model_checkpoint['state_dict'])
model.eval()

if torch.cuda.is_available():
    model = model.to('cuda')


In [3]:
# Define image transformations
preprocess = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize to match the model's input size
    transforms.ToTensor(),           # Convert to tensor
])

#### Accessing images

In [24]:
def protest_inference(path_to_img:str):
    # Load your input image
    image = Image.open(path_to_img)

    # Preprocess the image
    input_tensor = preprocess(image)

    # Add a batch dimension (1 image)
    input_tensor = input_tensor.unsqueeze(0)

    # Move all to cuda 
    if torch.cuda.is_available():
        input_tensor = input_tensor.to('cuda')

    # Perform inference
    with torch.no_grad():
        output = model(input_tensor)

    output = output.to('cpu')

    # Convert the output tensor to list
    output_list = output[0].tolist()
    
    return output_list

def run_detection_directory(path_to_res_file:str, country:str, test_data_dir: str):
    c = 0
    f = 0
    
    # File to store the results (parqur file)
    file_path = path_to_res_file+f'results_{country}.parquet'
    
    # Access the channel folder 
    channel_dirs = [filename for filename in os.listdir(test_data_dir) if not filename.startswith('.')]
    
    for channel_d in channel_dirs: # Get each channel 
        channel_name = channel_d
        
        sub_folder_path = os.path.join(test_data_dir, channel_d)
        
        # Access the video's frames folder 
        videos_dir = [filename for filename in os.listdir(sub_folder_path) if not filename.startswith('.')]
        
        for video_d in videos_dir:
            video_name = video_d
            
            video_path = os.path.join(sub_folder_path, video_d)
            
            frames = [f for f in os.listdir(video_path) if not f.startswith('.')] # Get the frames 
            
            for frame in frames:
                frame_path = os.path.join(video_path, frame)
                
                output_list = protest_inference(frame_path)
                
                # Extract country, channel name, video, and frame from the file path
                parts = frame_path.split('/')
                channel_name = parts[-3]
                video_name = parts[-2]
                frame_name = parts[-1]
                
                # Append the data to the list
                #data.append([channel_name, video_name, frame_name] + output_list)
            
                # Write the labels to a data frame
                aux = [channel_name, video_name, frame_name] + output_list
                data = [aux]
                df = pd.DataFrame(data, columns=['Channel', 'Video', 'Frame', "protest", "violence", "sign", "photo", "fire", "police", "children", "group_20", "group_100", "flag", "night", "shouting"])
                       
                # If the file does not exists, create it
                if not os.path.isfile(file_path): 
                    write(file_path, df)
                else: # Otherwise, write on it
                    write(file_path, df, append=True)
                
                f+=1
                if f>2:
                    break
            
            c +=1
            if c>1:
                break
                    
    
    return file_path

In [39]:
# Tree: sweden > channel > video > frames 

test_swe_path = '/zpool/beast-mirror/labour-movements-mobilisation-via-visual-means/youtube_video_frames/sweden/'
res_path = '/zpool/beast-mirror/labour-movements-mobilisation-via-visual-means/protest_derection_results/'
# Call the function and store the resulting DataFrame
#run_detection_directory(res_path, 'sweden', test_swe_path)

In [40]:
res = pd.read_parquet(res_path+'results_sweden.parquet', engine='fastparquet')

In [41]:
res.shape

(81330, 15)

### Processing results 

In [42]:
res_pro = res[["Channel", "Video", "Frame", "protest"]]

In [48]:
res_pro[res_pro['protest']>0.85].head(15)

,Channel,Video,Frame,protest
1069,TCOSverige,DYNK_B60m2A,DYNK_B60m2A_6750.jpg,0.963978
7583,sverigesarkitekter9502,JySPCVs1row,JySPCVs1row_5250.jpg,0.905296
7592,sverigesarkitekter9502,JySPCVs1row,JySPCVs1row_5500.jpg,0.927839
11574,Syndikalisterna,-a9oCTOWZQs,-a9oCTOWZQs_4640.jpg,0.856430
11586,Syndikalisterna,-a9oCTOWZQs,-a9oCTOWZQs_11020.jpg,0.903298
11591,Syndikalisterna,-a9oCTOWZQs,-a9oCTOWZQs_9280.jpg,0.945948
11593,Syndikalisterna,-a9oCTOWZQs,-a9oCTOWZQs_6670.jpg,0.884749
11597,Syndikalisterna,-a9oCTOWZQs,-a9oCTOWZQs_9860.jpg,0.905362
11602,Syndikalisterna,-a9oCTOWZQs,-a9oCTOWZQs_8410.jpg,0.908279
16122,akademikerforbundetssr,5HVANwYRHqk,5HVANwYRHqk_1500.jpg,0.929722


In [44]:
# Saving to csv file 
#res_pro.to_csv(res_path + 'protest_inference_sweden.csv') 

#### Check results from italy 

In [53]:
res_it = pd.read_parquet(res_path+'results_italy.parquet', engine='fastparquet')
res_it

,Channel,Video,Frame,protest,violence,sign,photo,fire,police,children,group_20,group_100,flag,night,shouting
0,UciIt,WSm7P6-Q0Lg,WSm7P6-Q0Lg_3000.jpg,0.013458,0.316697,0.729160,0.020539,0.056460,0.042030,0.021782,0.379616,0.040997,0.024737,0.023136,0.008337
1,UciIt,WSm7P6-Q0Lg,WSm7P6-Q0Lg_900.jpg,0.020906,0.289950,0.793661,0.015226,0.043836,0.018622,0.018096,0.383490,0.036750,0.022053,0.013438,0.004986
2,UciIt,WSm7P6-Q0Lg,WSm7P6-Q0Lg_2700.jpg,0.010393,0.303572,0.761524,0.018717,0.048136,0.037331,0.019623,0.390229,0.049158,0.026538,0.030408,0.007996
3,UciIt,WSm7P6-Q0Lg,WSm7P6-Q0Lg_600.jpg,0.024543,0.306502,0.729186,0.019662,0.066152,0.034656,0.027883,0.328730,0.033239,0.025478,0.016371,0.008071
4,UciIt,WSm7P6-Q0Lg,WSm7P6-Q0Lg_2400.jpg,0.010075,0.316211,0.744552,0.025616,0.081581,0.029809,0.019102,0.390043,0.050025,0.032795,0.029227,0.007636
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125,UciIt,eZv8WgMpZXE,eZv8WgMpZXE_9500.jpg,0.011464,0.281957,0.610054,0.052517,0.018159,0.032832,0.012751,0.674469,0.077793,0.466590,0.011333,0.012368
126,UciIt,eZv8WgMpZXE,eZv8WgMpZXE_1250.jpg,0.007364,0.271178,0.704699,0.035103,0.029493,0.016323,0.018044,0.357954,0.010204,0.040885,0.005791,0.008144
127,UciIt,eZv8WgMpZXE,eZv8WgMpZXE_4500.jpg,0.002448,0.208741,0.870397,0.028746,0.003243,0.004234,0.022962,0.658923,0.043552,0.050137,0.000815,0.003140
128,UciIt,eZv8WgMpZXE,eZv8WgMpZXE_500.jpg,0.003392,0.219398,0.609674,0.021870,0.017959,0.020668,0.045064,0.066996,0.004453,0.012015,0.016354,0.009518
